# General Setup

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
%pdb off

from pyCascade import probePost, physics, quantities
from matplotlib import pyplot as plt
import numpy as np
import os
from scipy.optimize import curve_fit
import pandas as pd
import matplotlib.colors as mcolors
from plotly import express as px

plt.rcParams['figure.dpi'] = 140
im_scaling = .75
plt.rcParams['figure.figsize'] = [6.4 * im_scaling, 4.8 * im_scaling]

############ Universal ################
scratch_home = os.getenv('SCRATCH') #need to set SCRATCH (even if there is no real SCRATCH) to the location where results are written
scratch_dir = f'{scratch_home}/Cascade/city_block_cfd'
home_dir = !pwd
home_dir = home_dir[0]

display(scratch_dir)
display(home_dir)
plt.close('all')



# Runs

In [2]:
runs = {
        253: {'A': 45, 'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        254: {'A': 45, 'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        255: {'A': 0,  'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        256: {'A': 0,  'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        321: {'A': 45, 'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        322: {'A': 45, 'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        323: {'A': 0,  'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        324: {'A': 0,  'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},

        246: {'A': 45, 'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        247: {'A': 45, 'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        248: {'A': 0,  'WS': 2, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        249: {'A': 0,  'WS': 4, 'C': 2, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        316: {'A': 45, 'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        318: {'A': 45, 'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        317: {'A': 0,  'WS': 2, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
        319: {'A': 0,  'WS': 4, 'C': 3, 'starts': [40000, 120000, 160000], 'stops': [119000, 159000, 240000], 'delT': [0, 5, 5], 'SS': [True, False, True]},
    }

plotFolder = f"{home_dir}/CHARLES/multiPlots/"

qoisOutputed = ["mag(u)_avg", "mag(u)_rms", "comp(u_rms,0)", "comp(u_rms,1)", "comp(u_rms,2)", "comp(u_avg,0)", "comp(u_avg,1)", "comp(u_avg,2)", "comp(u_rey,0)", "comp(u_rey,1)", "comp(u_rey,2)"]


In [ ]:
allABLVolStats = {}
overWrite = False
for run in runs:
    C = runs[run]['C']
    category = f"config{C}"
    R = run % 100
    probes_dir = f'{scratch_dir}/CHARLES/{category}/R{R}/probes/probesOut'
    oak_probes_dir =  f'{home_dir}/CHARLES/{category}/R{R}/probes/probesOut_parquet/'
    print(R)
    starts = runs[run]['starts']
    stops = runs[run]['stops']
    delT = runs[run]['starts']
    SS = runs[run]['SS']
    if len(set([len(starts), len(stops), len(delT), len(SS)])) != 1:
        raise Exception(f"Run {runs} has array properties of different lengths")

    probesLoaded = False
    for j, start in enumerate(starts):
        stop = stops[j]
        ABLVolStatsPath = f"{oak_probes_dir}/../ABLVolStats-{start}to{stop}.csv"
        paths = [ABLVolStatsPath]
        for path in paths:
            if os.path.exists(path) == False or overWrite:
                print(f"Creating stats for {run} from step {stop}. Saving to {path}...")
                if probesLoaded == False:
                    print(f"reading probes from {oak_probes_dir}")
                    file_type = "csv"
                    probes = probePost.Probes(probes_dir, directory_parquet = oak_probes_dir, file_type = file_type, probe_type = "VOLUMETRIC_PROBES", flux_quants = qoisOutputed, name_pattern = "VolProbe")
                    probesLoaded = True
                stats = probes.statistics(
                    names = probes.probe_names, 
                    steps = [stop],
                    quants = qoisOutputed,
                    processing = None,
                    parrallel=False
                )
                display(stats)
                stats.to_csv(ABLVolStatsPath)

        ABLVolStats = pd.read_csv(ABLVolStatsPath, index_col=0)
        for k, v in runs[run].items():
            if isinstance(v, list):
                ABLVolStats[k] = v[j]
            else:
                ABLVolStats[k] = v

        allABLVolStats[int(10*run + j)] = ABLVolStats
            


In [4]:
def processVolProbeNames(s, pos = None):
    # Filter out any characters that are not digits, a decimal point, or a negative sign
    filtered = ''.join(c for c in s if c.isdigit() or c == '.' or c == '_')
    filtered = filtered.split('_')
    
    # Convert the filtered string to float
    if filtered:
        if pos is not None:
            return float(filtered[1][pos])
        else:
            return float(''.join(filtered[0]))

In [5]:
ABLFits = pd.DataFrame(columns=["uStar", "z0", "disp"])
for i, (k, stats) in enumerate(allABLVolStats.items()):
    # stats = df.copy()
    A = stats["A"].iloc[0]
    A_rad = A * np.pi / 180
    stats["lowerBound"] = stats.index.map(lambda s: processVolProbeNames(s))
    stats["xPos"] = stats.index.map(lambda s: processVolProbeNames(s, pos=0))
    stats["zPos"] = stats.index.map(lambda s: processVolProbeNames(s, pos=1))
    stats.sort_values(by = ["xPos", "zPos", "lowerBound"], inplace=True)

    upperBound = stats["lowerBound"].copy()
    upperBound = upperBound.shift(-1)

    if stats["C"].iloc[0] == 2:
        H = 77.3
    elif stats["C"].iloc[0] == 3:
        H = 116
    else:
        raise Exception("Configuration {category} not an option")
    upperBound.fillna(H, inplace = True)
    upperBound[upperBound == min(stats["lowerBound"])] = H
    stats["upperBound"] = upperBound
    stats.dropna(inplace=True)
    stats["y"] = (stats["lowerBound"] + stats["upperBound"]) / 2

    stats['It^2'] = (stats['mag(u)_rms'] / stats['mag(u)_avg'])**2

    Uprime = stats['comp(u_rms,0)']
    Vprime = stats['comp(u_rms,1)']
    Wprime = stats['comp(u_rms,2)']
    Tau_02 = stats['comp(u_rey,1)']

    stats['I0^2'] = ((np.cos(A_rad)*Uprime)**2 + (np.sin(A_rad)*Wprime)**2 + 2*Tau_02*np.sin(A_rad)*np.cos(A_rad)) / stats['comp(u_avg,0)']**2
    stats['I1^2'] = (Vprime / stats['comp(u_avg,0)'])**2
    stats['I2^2'] = ((np.sin(A_rad)*Uprime)**2 + (np.cos(A_rad)*Wprime)**2 - 2*Tau_02*np.sin(A_rad)*np.cos(A_rad)) / stats['comp(u_avg,0)']**2
    stats['U/U10'] = stats["mag(u)_avg"] / stats["WS"]

ABLVolStatsMI = pd.concat(allABLVolStats.values(), keys=allABLVolStats.keys())
ABLVolStatsMI.index.set_names(['runs', 'probe'], inplace=True)

## Run Fits

In [ ]:
ABLVolStatsSSMI = ABLVolStatsMI[ABLVolStatsMI["SS"]==True]
ABLGroupedMI = ABLVolStatsSSMI.copy()
# ABLGroupedMI = ABLGroupedMI[ABLVolStatsMI["WS"]==2]
# ABLGroupedMI = ABLGroupedMI[ABLVolStatsMI["delT"]==5]
ABLGroupedMI = ABLGroupedMI.groupby(["C", "A", "y"]).mean()
ABLGroupedMI

In [ ]:
fig, axs = plt.subplots(2, 4, figsize = [6, 2.5], sharex =False, sharey = True)
fmts = ['o', 's', '^', 'v', '<', '>', 'D', 'X']
colors = list(mcolors.TABLEAU_COLORS)
fits = {}
for i, (k, df) in enumerate(ABLGroupedMI.groupby(level=[0,1])):
    df = df.reset_index()
    A = df["A"].iloc[0]
    C = df["C"].iloc[0]
    # WS = 2 #df["WS"].iloc[0]
    # delT = df["delT"].iloc[0]
    # row = 2*int(C==3)+int(A==45)
    row = int(C==3)
    ax = axs[row, 0]
    qty_vol = quantities.Qty()
    qty_vol.meanU = df["U/U10"]
    qty_vol.y = df['y'].values
    ax, fit = quantities.plot_ABL({f"{C}-{A}": qty_vol}, fit_disp=True, fitSlice = np.s_[6:], colorOffset=int(A==45), ax=ax, fmt = fmts[int(A==45)], linestyle='-', returnFit=True, markersize=1)
    fits[k] = fit
    # ax.set_title(f"{C}-{A}")
    ax.get_legend().remove()
    if row != 0:
        ax.set_xlabel("$U/U_{10}$")
    else:
        ax.set_xlabel("")
    ax.set_ylabel("y [m]")
    ax.set_xlim([0,2])

    scales = [1, .88, .55]
    symbols = ["u", "v", "w"]
    xmax = [2, 2, 1]
    for j, comp in enumerate([0, 1, 2]):
        scale = scales[j]
        ax = axs[row, 1+j]
        y = df['y'].values
        Iplot = np.sqrt(df[f"I{comp}^2"])
        ax.plot(Iplot, y, marker=fmts[int(A==45)], color = colors[int(A==45)], markersize=1, linewidth=0, label=f"{C}-{A}", fillstyle="none")
        z0 = fit['z0']
        d = fit['disp']
        y = y[y>3*z0+d]
        ax.plot(scale/np.log((y-d)/z0), y, color = colors[int(A==45)], linestyle='-', linewidth=.5)
        ax.legend()
        if row ==1:
            ax.set_xlabel(f"$I_{symbols[j]}$")
        ax.get_legend().remove()
        ax.set_xlim([0, xmax[j]])

plt.tight_layout()
pd.DataFrame(fits)

## Individual Fits

In [8]:
fmts = ['x', 'o']
styles = ['--', '-']
fits = {}
plt.close()
for i, (k, stats) in enumerate(allABLVolStats.items()):
    A = stats["A"].iloc[0]
    C = stats["C"].iloc[0]
    WS = stats["WS"].iloc[0]
    delT = stats["delT"].iloc[0]
    name = f"{k}: {C}-{A}|{WS}-{delT}"
    ax.set_title(name)
    theta = stats["A"].iloc[0]
    thetaRad = theta / 180 * np.pi
    meanU = np.cos(thetaRad) * stats["comp(u_avg,0)"] + np.sin(thetaRad) * stats["comp(u_avg,2)"]
    statsMI = stats.set_index(["xPos", "zPos", "y"])
    c = 0
    fmt = fmts[int(k%2)]
    style = styles[int(k%2)]
    for xPos, dfx in statsMI.groupby(level=0):
        for zPos, df in dfx.groupby(level=1):
            qty_vol = quantities.Qty()
            # meanU = np.cos(thetaRad) * df["comp(u_avg,0)"] + np.sin(thetaRad) * df["comp(u_avg,2)"]
            meanU = df["mag(u)_avg"]
            # meanU = np.arctan(df["comp(u_avg,2)"] / df["comp(u_avg,0)"]) * 180 / np.pi
            qty_vol.meanU = meanU.iloc[0:18]#[6:27]
            qty_vol.y = df.index.get_level_values(2).values[0:18]
            if A == 45:
                id = int((xPos-zPos)%4)
            elif A == 0:
                id = int(zPos)
            ax, fit = quantities.plot_ABL({f"{xPos}-{zPos}: {id}": qty_vol}, fit_disp=True, fitSlice = np.s_[3:], colorOffset=id, ax=ax, fmt = fmt, linestyle=style, returnFit=True)
            fit["id"] = id
            fit["A"] = A
            fit["C"] = C
            fit["WS"] = WS
            fit["delT"] = delT
            fit["xPos"] = xPos
            fit["zPos"] = zPos
            fits[(k, xPos, zPos)] = fit
            c += 1

fits = pd.DataFrame(fits).T


In [ ]:
def adjusted_friction_velocity(ustar1, z01, z02):
    """
    Calculate the friction velocity u*,2 over the second surface using the friction velocity u*,1 over the first surface,
    and the roughness lengths of both surfaces.
    
    Parameters:
    ustar1 : float
        Friction velocity over the first surface (u*,1).
    z01 : float
        Roughness length of the first surface (z0,1).
    z02 : float
        Roughness length of the second surface (z0,2).
        
    Returns:
    float
        The friction velocity over the second surface (u*,2).
    """
    ustar2 = ustar1 * (z02 / z01) ** 0.07
    return ustar2

def label_domain_positions(x, z):
    domainSplit = 1
    if x > domainSplit and z > domainSplit:
        return 'B'
    elif x > domainSplit and z <= domainSplit:
        return 'Bz'
    elif x <= domainSplit and z > domainSplit:
        return 'Bx'
    elif x <= domainSplit and z <= domainSplit:
        return 'Bxz'
    else:
        raise Exception("Coordinates not mapped to labels")

In [8]:
fits.index.names = ["runs", "xPosInd", "zPosInd"]
fits["blockType"] = fits.apply(lambda row: label_domain_positions(row['xPos'], row['zPos']), axis=1)

In [9]:
fits["uStarNorm"] = fits["uStar"] / fits["WS"]

In [ ]:
px.scatter_3d(fits[fits["A"] == 0], "xPos", "zPos", "uStarNorm", color = "C")

In [ ]:
hm = 6
z_0 = 0.061*hm
display(fit, z_0)
fits["uStar_2"]= adjusted_friction_velocity(fits["uStar"], fits["z0"], z_0)
fits["z0_2"] = z_0
fits["u10_2"] = physics.loglaw_with_disp(np.array([10]), fits["uStar_2"], z_0, 0)
fits["uScaling"] = fits["u10_2"] / fits["WS"]
px.scatter_3d(fits[fits["A"] == 0], "xPos", "zPos", "uScaling", color = "C")

In [ ]:
px.scatter_3d(fits[fits["A"] == 0], "xPos", "zPos", "z0", color = "C")

In [ ]:
px.scatter_3d(fits[fits["A"] == 45], "xPos", "zPos", "uScaling", color = "C")

In [ ]:
px.scatter_3d(fits[fits["A"] == 45], "xPos", "zPos", "z0", color = "C")

In [ ]:
blockFits = fits.groupby(['runs', 'blockType'], as_index=True).mean()
px.scatter_3d(blockFits[blockFits["A"] == 0], "xPos", "zPos", "uScaling", color = "C")

In [ ]:
px.scatter_3d(blockFits[blockFits["A"] == 0], "xPos", "zPos", "z0", color = "C")

In [ ]:
px.scatter_3d(blockFits[blockFits["A"] == 45], "xPos", "zPos", "uScaling", color = "C")

In [ ]:
px.scatter_3d(blockFits[blockFits["A"] == 45], "xPos", "zPos", "z0", color = "C")

In [ ]:
px.scatter(fits.reset_index(), "blockType", "uScaling", color = "A", symbol="C")

In [20]:
fits.to_csv(f"{home_dir}/CHARLES/multiRuns/fitsABL.csv")
blockFits.to_csv(f"{home_dir}/CHARLES/multiRuns/blockFitsABL.csv")
ABLVolStatsMI.to_csv(f"{home_dir}/CHARLES/multiRuns/ABLVolStatsMI.csv")